# SGGSLog Tutorial

This notebook demonstrates the core features of SGGSLog, a logic programming system based on Semantically Guided Goal-Sensitive reasoning (SGGS).

SGGSLog allows you to:
- Define facts and rules in first-order logic
- Query the knowledge base with pattern matching
- Handle recursive definitions (like graph reachability)
- Work with disjunctive information
- Use quantifiers for expressive power

## Section 1: Basic Facts and Queries

Let's start with the basics. We can define **ground facts** - simple statements about the world.

In [ ]:
person alice

In [ ]:
person bob

In [ ]:
person carol

Now we can **query** our knowledge base. A query starts with `?-` and asks whether something is true.

In [ ]:
?- person alice

We can use **variables** (uppercase identifiers like `X`) to find all matching values:

In [ ]:
?- person X

Use `:next` to get additional answers from the query stream:

In [ ]:
:next

In [ ]:
:next

In [ ]:
:next

## Section 2: Rules and Implications

**Rules** let us derive new facts from existing ones. The syntax `A -> B` means "if A then B".

In [ ]:
person X -> mortal X

Now SGGS can derive that alice, bob, and carol are all mortal:

In [ ]:
?- mortal alice

In [ ]:
?- mortal X

In [ ]:
:next

In [ ]:
:next

## Section 3: Recursive Rules (Graph Reachability)

One of SGGS's strengths is handling **recursive definitions**. Let's define a graph and compute reachability.

In [ ]:
edge a b

In [ ]:
edge b c

In [ ]:
edge c d

Now we define path reachability recursively:
1. Direct edges create paths
2. Paths are transitive

In [ ]:
edge X Y -> path X Y

In [ ]:
path X Y & edge Y Z -> path X Z

Query: What nodes can we reach from `a`?

In [ ]:
?- path a X

In [ ]:
:next

In [ ]:
:next

In [ ]:
:next

## Section 4: Universal Quantifiers

SGGSLog supports full first-order logic, including **universal quantification** with `∀`.

In [ ]:
∀X ∀Y (path X Y -> connected X Y)

In [ ]:
?- connected a d

## Section 5: Disjunctive Reasoning

Unlike Prolog, SGGS handles **disjunction** natively. This means we can reason about alternatives.

In [ ]:
item x1

In [ ]:
item x2

In [ ]:
item X -> red X | blue X

This says every item is either red or blue (but we don't know which). SGGS will reason about both possibilities.

In [ ]:
?- red x1

## Section 6: Existential Quantifiers and Projection

**Existential quantification** (`∃`) asserts that something exists without naming it. SGGSLog creates an internal witness (Skolem constant).

In [ ]:
∃S (secret S)

By default, SGGSLog hides internal symbols in query answers to keep output clean:

In [ ]:
?- secret X

We can change the projection setting to reveal internal symbols:

In [ ]:
:set projection allow_internal

In [ ]:
?- secret X

## Section 7: Compound Terms and Arithmetic

Logic programming's real power comes from **compound terms** - structured data that can be constructed and deconstructed through unification.

We represent natural numbers using **Peano numerals**:
- `z` is zero
- `(s N)` is the successor of N (i.e., N + 1)

So: `z` = 0, `(s z)` = 1, `(s (s z))` = 2, `(s (s (s z)))` = 3, etc.

We define addition as a ternary relation where `add X Y Z` means X + Y = Z:

In [1]:
add z Y Y

ok.

In [1]:
add X Y Z -> add (s X) Y (s Z)

ok.

In [1]:
?- add A B C

A = z, B = Y, C = Y

In [1]:
:next

B = Y, A = (s (s (s (s (s (s (s z))))))), C = (s (s (s (s (s (s (s Y)))))))

The first rule says: 0 + Y = Y (base case).

The second rule says: if X + Y = Z, then (X+1) + Y = (Z+1) (recursive case).

Now we can **compute addition** by querying with the result unknown:

In [ ]:
?- add (s z) (s (s z)) R

That computes 1 + 2 = R, giving R = 3 (as `(s (s (s z)))`).

The magic of logic programming: the **same relation works backwards**. We can compute subtraction by querying with the first argument unknown:

In [ ]:
?- add X (s z) (s (s (s z)))

That computes X + 1 = 3, giving X = 2 (as `(s (s z))`). This is subtraction!

We can even **enumerate all partitions** of a number:

In [1]:
?- add X Y (s (s z))

X = z, Y = (s (s z))

In [1]:
:next

Y = (s z), X = (s z)

In [1]:
:next

Y = z, X = (s (s z))

In [1]:
:next

timeout.

In [1]:
:stats

pending_answers=0, seen_answers=3, steps_taken=47, derivation_done=true

This finds all X, Y such that X + Y = 2: (0,2), (1,1), (2,0).

This bidirectional use of relations is what makes logic programming fundamentally different from functional programming.

## Section 8: Unsatisfiability Detection

SGGS can detect **contradictions** in a theory. When a theory is unsatisfiable, SGGS derives the empty clause.

Let's add a contradiction:

In [1]:
contra

ok.

In [1]:
~contra

ok.

Now any query will reveal the theory is unsatisfiable:

In [ ]:
?- anything

In [1]:
:set timeout_ms 200000

Set timeout_ms = 200000

## Summary

This tutorial covered:

1. **Facts** - Ground statements (`person alice`)
2. **Queries** - Pattern matching with variables (`?- person X`)
3. **Query Streaming** - Getting multiple answers with `:next`
4. **Rules** - Implications (`person X -> mortal X`)
5. **Recursive Rules** - Transitive closure (`path X Y & edge Y Z -> path X Z`)
6. **Universal Quantifiers** - `∀X (...)` patterns
7. **Disjunction** - Non-deterministic choices (`red X | blue X`)
8. **Existential Quantifiers** - `∃X (...)` with Skolemization
9. **Projection** - Controlling visibility of internal symbols
10. **Compound Terms** - Structured data like `(s (s z))`
11. **Bidirectional Relations** - Using `add` for both addition and subtraction
12. **Unsatisfiability** - Detecting contradictions